## Modelo de datos de ofertas de tarjetas de crédito
#### Este modelo de datos es utilizado por una aplicación de ofertas de tarjetas de crédito.

Un proveedor de tarjeta de crédito produce ofertas a lo largo del tiempo. Estas ofertas incluyen transferencias de saldo sin cargos, mayores límites de crédito, tasas de interés más bajas, devolución de efectivo y millas aéreas. Después de que un cliente acepte o rechace estas ofertas, el estado de la oferta correspondiente se actualiza en consecuencia.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de registros de cuenta usando AccountId, según lo facilitado en la tabla principal.

* Recuperación de todas las cuentas con pocos elementos previstos, facilitada por el índice secundario AccountIndex.

* Recuperación de cuentas y todos los registros de oferta asociados con esas cuentas mediante el uso de AccountId, facilitado por la tabla principal.

* Recuperación de cuentas y registros de ofertas específicas asociadas con esas cuentas mediante el uso de AccountId y OfferId, según lo facilitado en la tabla principal.

* Recuperación de todos los registros de oferta ACCEPTED/DECLINED de OfferType específicos asociados con cuentas usando AccountId, OfferType, y Status, según lo facilitado por el índice secundario GSI1.

* Recuperación de ofertas y registros de elementos de oferta asociados utilizando OfferId, según lo facilitado en la tabla principal.

TABLE:
<br>

![image.png](images/credit_card01.png)

GSI AccountIndex:
<br>

![image.png](images/credit_card02.png)

GSI GSI1:
<br>

![image.png](images/credit_card03.png)

In [1]:
import pandas as pd
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from time import sleep
import boto3

In [ ]:
dt = DynamoTable(profile_name="my-profile")
try:
    dt.select_table('CreditCardOffers')
    print(dt)
except:
    dt.create_table(
        table_name='CreditCardOffers',
        partition_key='PK',
        partition_key_type='S',
        sort_key="SK",
        sort_key_type="S"
)

In [ ]:
dt.load_json("credits.json")

In [ ]:
dt.create_global_secondary_index(
    att_name="AccountIndexId",
    att_type="S",
    i_name="AccountIndex",
    proj_type=["FirstName", "MiddleName", "LastName", "Emailid"],
    i_rcu=1,
    i_wcu=1
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [ ]:
dt.create_global_secondary_index(
    att_name="PK",
    att_type="S",
    sort_index="GSI1SK",
    sort_type="S",
    i_name="GSI2",
    provisioned=False
)

In [ ]:
status = dt.check_status_gsi()
while status == 'CREATING':
    status = dt.check_status_gsi()
    sleep(30)
print("Global secondary index created.")

In [5]:
dt.list_gsi

['GSI2', 'AccountIndex', 'GSI1']

In [ ]:
dt.query_partiql(
    query="SELECT * FROM CreditCardOffers WHERE Status = ?",
    parameters=['CREATED']
)